In [ ]:
import sys
print(sys.executable)

#& "C:\Program Files\Python312\python.exe" -m pip install pdfplumber
#& "C:\Program Files\Python312\python.exe" -m pip install camelot
#& "C:\Program Files\Python312\python.exe" -m pip install camelot-py[cv] pdfplumber pandas openpyxl

c:\Program Files\Python312\python.exe


In [12]:
import os
import re
import pdfplumber
import pandas as pd
# Directory containing uploaded PDFs

INPUT_DIR  =  r'C:\Users\ijttr\OneDrive\Dokumenty\PROG\PYTHON\DATA_ANALYSIS\PD'
OUTPUT_DIR = INPUT_DIR
os.makedirs(OUTPUT_DIR, exist_ok=True)



# Regex: kód = písmeno + možná tečka/pomlčka + 1–3 číslic
# Descr = 1–100 znaků (lenivě), pak číslo s desetinnou čárkou/tečkou
PATTERN = re.compile(
    r'([A-Z])[.\-]?\s*(\d{1,3})'      # 1: sekce, 2: číslo
    r'[\s–\-:]*'                      # možný oddělovač
    r'(.{5,100}?)'                    # 3: popis místnosti (5–100 znaků)
    r'(\d{1,4}[.,]\d{1,2})'           # 4: výměra
    , re.DOTALL | re.IGNORECASE
)

def extract_from_pdf(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for p in pdf.pages:
            text += p.extract_text() or "" + "\n"
    matches = PATTERN.finditer(text)
    rows = []
    for m in matches:
        sect, num, desc, area = m.groups()
        code = f"{sect.upper()}.{int(num):03d}"
        desc = re.sub(r'\s+', ' ', desc).strip()
        area_f = float(area.replace(',', '.'))
        rows.append([code, desc, area_f])
    return rows

def process_pdf(pdf_path):
    name = os.path.splitext(os.path.basename(pdf_path))[0]
    rows = extract_from_pdf(pdf_path)
    if not rows:
        print(f"⚠️ V {name}.pdf jsem nic nenašel.")
        return
    
    df = pd.DataFrame(rows, columns=['Číslo místnosti','Účel místnosti','Výmera (m²)'])
    
    # Seřazení
    df[['Sekce','Číslo']] = df['Číslo místnosti'].str.split('.', expand=True)
    df['Číslo'] = df['Číslo'].astype(int)
    df = df.sort_values(['Sekce','Číslo']).drop(columns=['Sekce','Číslo'])
    
    # Desetinná čárka
    df['Výmera (m²)'] = df['Výmera (m²)'].map(lambda x: f"{x:.1f}".replace('.', ','))
    
    out = os.path.join(OUTPUT_DIR, f"{name}.xlsx")
    df.to_excel(out, index=False)
    print(f"✅ Uloženo: {out}")

if __name__ == '__main__':
    for fn in os.listdir(INPUT_DIR):
        if fn.lower().endswith('.pdf'):
            process_pdf(os.path.join(INPUT_DIR, fn))
    print("Hotovo – Excely jsou ve složce:", OUTPUT_DIR)



CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


✅ Uloženo: C:\Users\ijttr\OneDrive\Dokumenty\PROG\PYTHON\DATA_ANALYSIS\PD\dur_nkb_d01_01_4-3pp_230600.xlsx


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


✅ Uloženo: C:\Users\ijttr\OneDrive\Dokumenty\PROG\PYTHON\DATA_ANALYSIS\PD\dur_nkb_d01_02_2pp_230600.xlsx


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


✅ Uloženo: C:\Users\ijttr\OneDrive\Dokumenty\PROG\PYTHON\DATA_ANALYSIS\PD\dur_nkb_d01_03_1pp_230600.xlsx


CropBox missing from /Page, defaulting to MediaBox


✅ Uloženo: C:\Users\ijttr\OneDrive\Dokumenty\PROG\PYTHON\DATA_ANALYSIS\PD\dur_nkb_d01_04_a1pp_230600.xlsx
Hotovo – Excely jsou ve složce: C:\Users\ijttr\OneDrive\Dokumenty\PROG\PYTHON\DATA_ANALYSIS\PD
